In [6]:
from IPython.display import display, HTML

# Apply word wrapping to notebook outputs
display(HTML('''
    <style>
        .output_area pre {
            white-space: pre-wrap;
        }
    </style>
'''))

import json
import pandas as pd
import time
from openai import OpenAI
import os
import ast
import numpy as np
import pdb
from dotenv import load_dotenv
from pprint import pprint
from openai import RateLimitError, AuthenticationError, APIConnectionError, APIError

# Load environment variables from .env file
load_dotenv()

# Initialize OpenAI client with API key from environment variable
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def get_response(user_content:str, system_content:str=None, assistant_content:str=None):
    try:
        messages = [{"role": "user", "content": user_content}]

        if system_content is not None:
            messages.append({"role": "system", "content": system_content})

        if assistant_content is not None:
            messages.append({"role": "assistant", "content": assistant_content})

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        return response
    except RateLimitError as e:
        print(f"Rate limit exceeded: {e}")
        # print("Retrying after 60 seconds...")
        # time.sleep(60)  # Wait for 60 seconds before retrying
        # return get_response(question)
        # return response
    except APIConnectionError as e:
        print(f"API connection error: {e}")
    except AuthenticationError as e:
        print(f"Authentication error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


def get_embedding(text, model="text-embedding-3-small"):
    try:
        text = text.replace("\n", " ")
        response = client.embeddings.create(
            model=model,
            input=[text]
        )
        return response.data[0].embedding
    except RateLimitError as e:
        print(f"Rate limit exceeded: {e}")
        # print("Retrying after 60 seconds...")
        # time.sleep(60)  # Wait for 60 seconds before retrying
        # return get_response(question)
        # return response
    except APIConnectionError as e:
        print(f"API connection error: {e}")
    except AuthenticationError as e:
        print(f"Authentication error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [5]:
json_data = json.load(open("data/insurance-rag.json"))
# Function to concatenate relevant fields for service agents
def format_service_agent(agent):
    return f"Agent ID: {agent['agent_id']}, Name: {agent['name']}, " \
           f"Expertise: {', '.join(agent['area_of_expertise'])}, " \
           f"Experience: Health - {agent['experience_handling_health_claims']} years, " \
           f"Auto - {agent['experience_handling_auto_claims']} years, " \
           f"Property - {agent['experience_handling_property_claims']} years, " \
           f"Overall - {agent['overall_experience']} years."

# Function to concatenate relevant fields for claims
def format_claim(claim):
    return f"Claim ID: {claim['claim_id']}, Customer: {claim['customer_name']} ({claim['customer_id']}), " \
           f"Insurance Type: {claim['insurance_type']}, Policy Number: {claim['policy_number']}, " \
           f"Incident Date: {claim['incident_date']}, Location: {claim['incident_location']}, " \
           f"Description: {claim['incident_description']}, Claim Type: {claim['claim_type']}, " \
           f"Claim Amount: ${claim['claim_amount']}, Feedback: {claim['claimant_feedback']}"

# Format the data
formatted_agents = [format_service_agent(agent) for agent in json_data['service_agents']]
formatted_claims = [format_claim(claim) for claim in json_data['claims']]
formatted_agents, formatted_claims, '...'

(['Agent ID: a1, Name: John Smith, Expertise: health, auto, Experience: Health - 7 years, Auto - 6 years, Property - 4 years, Overall - 6 years.',
  'Agent ID: a2, Name: Emily Johnson, Expertise: property, Experience: Health - 2 years, Auto - 3 years, Property - 8 years, Overall - 4 years.',
  'Agent ID: a3, Name: Michael Davis, Expertise: auto, property, Experience: Health - 3 years, Auto - 8 years, Property - 7 years, Overall - 6 years.',
  'Agent ID: a4, Name: Jessica Williams, Expertise: health, Experience: Health - 9 years, Auto - 1 years, Property - 2 years, Overall - 4 years.',
  'Agent ID: a5, Name: David Brown, Expertise: health, auto, property, Experience: Health - 6 years, Auto - 7 years, Property - 5 years, Overall - 6 years.',
  'Agent ID: a6, Name: Sophia Martinez, Expertise: auto, Experience: Health - 2 years, Auto - 9 years, Property - 3 years, Overall - 5 years.',
  'Agent ID: a7, Name: James Anderson, Expertise: health, property, Experience: Health - 8 years, Auto - 2

In [15]:
def get_claims_context(formatted_claims):
    formatted_str = "Here are the details of the past claims:\n\n"
    for claim in formatted_claims:
        formatted_str += claim + "\n\n"
    return formatted_str

claims_context = get_claims_context(formatted_claims)
claims_context

'Here are the details of the past claims:\n\nClaim ID: C001, Customer: John Doe (cust001), Insurance Type: Health, Policy Number: P10001, Incident Date: 2024-08-15, Location: XYZ Hospital, Description: Treatment for broken arm after a fall., Claim Type: Medical Expense, Claim Amount: $2000, Feedback: The process was very smooth. Great service.\n\nClaim ID: C002, Customer: John Doe (cust001), Insurance Type: Auto, Policy Number: P10002, Incident Date: 2024-09-10, Location: 5th Avenue, NY, Description: Minor collision at an intersection., Claim Type: Collision Damage, Claim Amount: $5000, Feedback: The agent was a bit slow to respond.\n\nClaim ID: C003, Customer: John Doe (cust001), Insurance Type: Property, Policy Number: P10003, Incident Date: 2024-07-22, Location: 123 Elm St, Description: Roof damage due to a storm., Claim Type: Weather Damage, Claim Amount: $8000, Feedback: Neutral experience, nothing remarkable.\n\nClaim ID: C004, Customer: Jane Smith (cust002), Insurance Type: Heal

In [16]:
def get_agent_context(formatted_agents):
    formatted_str = "Here are the details of the claim processing agents:\n\n"
    for claim in formatted_agents:
        formatted_str += claim + "\n\n"
    return formatted_str

agents_context = get_agent_context(formatted_agents)
agents_context

'Here are the details of the claim processing agents:\n\nAgent ID: a1, Name: John Smith, Expertise: health, auto, Experience: Health - 7 years, Auto - 6 years, Property - 4 years, Overall - 6 years.\n\nAgent ID: a2, Name: Emily Johnson, Expertise: property, Experience: Health - 2 years, Auto - 3 years, Property - 8 years, Overall - 4 years.\n\nAgent ID: a3, Name: Michael Davis, Expertise: auto, property, Experience: Health - 3 years, Auto - 8 years, Property - 7 years, Overall - 6 years.\n\nAgent ID: a4, Name: Jessica Williams, Expertise: health, Experience: Health - 9 years, Auto - 1 years, Property - 2 years, Overall - 4 years.\n\nAgent ID: a5, Name: David Brown, Expertise: health, auto, property, Experience: Health - 6 years, Auto - 7 years, Property - 5 years, Overall - 6 years.\n\nAgent ID: a6, Name: Sophia Martinez, Expertise: auto, Experience: Health - 2 years, Auto - 9 years, Property - 3 years, Overall - 5 years.\n\nAgent ID: a7, Name: James Anderson, Expertise: health, proper

In [19]:
system_prompt = '''
You are an AI assistant responsible for selecting the most suitable insurance agent to handle a new customer claim. You will be provided with:

1. A chat history between a customer and the help desk regarding the claim, which will be included in the user's prompt.
2. Claim details including The customer id, their past claim history, and the available claim details.
3. A list of insurance agents, including their expertise and experience in handling various insurance types.

Your task is to analyze the conversation and claim details, and based on the customer's needs and the agents' expertise, recommend the most suitable agent to handle the claim.

Consider the following factors when choosing the agent:

1. Insurance type of the claim.
2. The complexity and urgency of the claim.
3. The agent’s expertise and experience in handling similar claims.'''
system_prompt

"\nYou are an AI assistant responsible for selecting the most suitable insurance agent to handle a new customer claim. You will be provided with:\n\n1. A chat history between a customer and the help desk regarding the claim.\n2. The customer’s ID, their past claim history, and the available claim details.\n3. A list of insurance agents, including their expertise and experience in handling various insurance types.\n\nYour task is to analyze the conversation and claim details, and based on the customer's needs and the agents' expertise, recommend the most suitable agent to handle the claim.\n\nConsider the following factors when choosing the agent:\n\n1. Insurance type of the claim.\n2. The complexity and urgency of the claim.\n3. The agent’s expertise and experience in handling similar claims."

In [21]:
assistant_prompt = f'''
You have been provided with the following information: {claims_context}, {agents_context}
'''
assistant_prompt

'\nYou have been provided with the following information: Here are the details of the past claims:\n\nClaim ID: C001, Customer: John Doe (cust001), Insurance Type: Health, Policy Number: P10001, Incident Date: 2024-08-15, Location: XYZ Hospital, Description: Treatment for broken arm after a fall., Claim Type: Medical Expense, Claim Amount: $2000, Feedback: The process was very smooth. Great service.\n\nClaim ID: C002, Customer: John Doe (cust001), Insurance Type: Auto, Policy Number: P10002, Incident Date: 2024-09-10, Location: 5th Avenue, NY, Description: Minor collision at an intersection., Claim Type: Collision Damage, Claim Amount: $5000, Feedback: The agent was a bit slow to respond.\n\nClaim ID: C003, Customer: John Doe (cust001), Insurance Type: Property, Policy Number: P10003, Incident Date: 2024-07-22, Location: 123 Elm St, Description: Roof damage due to a storm., Claim Type: Weather Damage, Claim Amount: $8000, Feedback: Neutral experience, nothing remarkable.\n\nClaim ID: C